Question 1: Scraping the Toronto post codes table from Wikipedia and getting it into a usable pandas format

Start with importing the libraries we'll need for the exercise

In [64]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from pathlib import Path

Run the beautiful soup method on the url to get the data in a JSON format

In [65]:
broth = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bs(broth)

Now pull the data just from tables 

In [66]:
table = soup.find('table',{'class':'wikitable sortable'})
cols = table.findAll('td')

Strip out the html formatting so that we end up with just strings

In [67]:
dataRaw = []
df = pd.DataFrame()
for i in range(len(cols)):
    dataRaw.append(cols[i].text.strip())

We pulled the data into a single list, but now we nead to split it up into appropriate columns. I'm using a nested iterator to cycle from 0 to 2 to fill in the columns. So we'll do that, then assign the columns to a new dataframe with Pandas. 
<br><br>If you're smarter than me though, you either found a better way to do this, or didn't have this issue to start with

In [68]:
col1 = []
col2 = []
col3 = []
j = 0
for i in range(len(cols)):
    if j == 3:
        j = 0
        col1.append(dataRaw[i])
    elif j == 0:
        col1.append(dataRaw[i])
    elif j == 1: 
        col2.append(dataRaw[i])
    elif j == 2:
        col3.append(dataRaw[i])
    else:
        raise Exception('You done messed up the looping') 
    j += 1
df = pd.DataFrame()
df['Post Code'] = col1
df['Borough'] = col2
df['Neighborhood'] = col3
df.head()

,Post Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Whew boi, looks like there's some post codes that aren't assigned. Let's count those up and strip them out to clean up our dataframe

In [69]:
oldCount = df.shape
notAssignedMask = df['Borough'] == 'Not assigned'
keepMask = ~notAssignedMask
df = df[keepMask]
df.shape, oldCount

((212, 3), (289, 3))

Cool, so we stripped out 80 or so entries in the dataframe. 
<br><br>With that, I think it's time to get down to business. Let's check duplicates just to be sure

In [70]:
df.groupby('Post Code').nunique()

,Post Code,Borough,Neighborhood
Post Code,,,
M1B,1,1,2
M1C,1,1,3
M1E,1,1,3
M1G,1,1,1
M1H,1,1,1
M1J,1,1,1
M1K,1,1,3
M1L,1,1,3
M1M,1,1,3


Ah dang it, there's multiple neighborhoods per post-code. 
<br><br>Use groupby.agg to aggregate the neighborhoods based on the Post Code, separate them all using a , punctuation

In [71]:
test = df.copy()
new = test.groupby('Post Code').agg({'Borough' :'first',
                                     'Neighborhood': ', '.join,
                                     }).reset_index()
df = new.copy()
df.shape

(103, 3)

In [72]:
latLong = pd.read_csv('/Users/chrismay/Desktop/ibmDSCapstone/Geospatial_Coordinates.csv')
latLong = latLong.astype(object)
new = pd.concat([df, latLong], axis = 1, join = 'inner')
df = new.copy()

In [73]:
df.drop(['Postal Code'], axis = 1, inplace = True)

In [75]:
df.head(12)

,Post Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111,-79.2846
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6927,-79.2648
